# Body of the code

This notebook is the full pipeline of the GPU-accelerated integrator and the CPU autocorrelation function calculator (+ plots).

In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

import numba
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_normal_float32

## Parameters

In [ ]:
M = 1_000         #number of trajectories

N = 1_000_000   #number of integration steps

dt = 2e-3       #integration step size

## Potential

In [ ]:
# Device function (runs on GPU only) - this will inform the CUDA simulations
@cuda.jit(device=True)
def device_Vprime(x, a):
  if x>a: return 3.0*(x-(a+2.0))*(x-a)**2 + (x-a)**3
  elif x<-a: return 3.0*(x+(a+2.0))*(x+a)**2 + (x+a)**3
  else: return 2.0/(5.0*a**2)*x

## RNG

In [ ]:
host_seed = 2**63 - 1
# This is necessary for the xoroshiro generator

## Simulation Functions

In [ ]:
def launch_EMOL_M32(a=2.0, N=25_000, M=200, dt=np.single(1e-3), eps=0.5, blockdim=256, ic=0.0, rngseed=host_seed):
  griddim = int(np.ceil(M/blockdim))
  host_X = np.empty((M,N), dtype=np.single)
  host_X[:,0] = ic
  dev_X = cuda.to_device(host_X)
  a = np.single(a)

  rng_states = create_xoroshiro128p_states(M, seed=rngseed) # necessary for device rng
  sqrt2epsdt = np.single(np.sqrt(2*eps*dt))

  kernel_update_traj32_devrng[griddim, blockdim](dev_X, a, N, M, dt, sqrt2epsdt, rng_states)
  cuda.synchronize()

  host_X = dev_X.copy_to_host()
  return host_X


@cuda.jit
def kernel_update_traj32_devrng(dev_X, a, N, M, dt, sqrt2epsdt, rng_states):
  glob_idx = cuda.grid(1)
  if glob_idx >= M:
    return
  x = dev_X[glob_idx, 0]
  for i in range(N-1):
    noise = xoroshiro128p_normal_float32(rng_states, glob_idx)
    x = kernel_1stepEMOL(x, a, sqrt2epsdt, dt, noise)
    dev_X[glob_idx, i+1] = x


@cuda.jit(device=True)
def kernel_1stepEMOL(x, a, sqrt2epsdt, dt, noise):
  drift = -device_Vprime(x, a) * dt
  diff = sqrt2epsdt * noise
  x += drift + diff
  return x

## Running Simulations

In [ ]:
X = launch_EMOL_M32(a=2.0, M=M, N=N, dt=dt, blockdim=128)

del X[:5000] # Burn first 5000 steps - this is to ensure relaxation to equilibrium
# a better (read: more rigorous) way of doing this is to initially sample from the
# pdf, since we already know it

## Building the ACF

In [ ]:
def acovs_WK_unbiased(X): # host function
  acovs = np.empty((np.shape(X)))
  for i in range(np.shape(X)[0]):
    acovs[i] = traj_acov_WK_unbiased(X[i])
  return acovs

def traj_acov_WK_unbiased(traj): # host function
  x = traj - np.mean(traj)
  n = len(x)
  npad = 1 << (2*n-1).bit_length()
  Fx = np.fft.rfft(x, n=npad)
  ps = Fx * np.conjugate(Fx) # see Weiner Khinchin
  acov = np.fft.irfft(ps, n=npad)
  acov = acov[:n]
  count = np.arange(n, 0, -1)
  return acov/count

# Plots

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1,2,1)
plt.hist(X.flatten(), bins=800, density=True, alpha=0.7, align="mid")
plt.xlabel("x")
plt.ylabel("Density")
plt.title("Histogram of burned X values")

plt.subplot(1,2,2)
mask = t < t[len(t)//10] #[to prevent noise]
plt.plot(t[mask], acf[mask], label="ACF", lw=2.0)
plt.fill_between(t[mask],
                 acf[mask] + acf_sd[mask],
                 acf[mask] - acf_sd[mask], alpha=0.25, label="sd")
plt.xlabel("Time lag")
plt.ylabel("Normalised autocorrelation")
plt.title("Ensemble avg. ACF")
plt.legend()
plt.grid()
plt.tight_layout()

plt.show()